<a href="https://colab.research.google.com/github/LarsGart/NLP-Social-Media-Mining-for-Health-Monitoring/blob/experimental/Task%202/Task_2_Data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install demoji
!pip install transformers 
!pip install pytorch-lightning

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import demoji
# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text

In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP-Social-Media-Mining-for-Health-Monitoring/datasets/Task 2/task3_training.tsv', sep = '\t')
df_eval = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP-Social-Media-Mining-for-Health-Monitoring/datasets/Task 2/task3_validation.tsv', sep = '\t')
df_eval

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,@kyleecakes4 i was given like 5mg of abilify a...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN


In [5]:
df_1 = df[df['type'] == "ADR"]
df_0 = df[df['type'] != "ADR"]
df_1

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,Unnamed: 9,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
27,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10047522.0,vision loss,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10048010.0,withdrawal syndrome,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospi...,10028823.0,nauseous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10021574.0,inability to orgasm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10047896.0,weight gain,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,347414282138431488,38.0,42.0,ADR,flat,zyprexa,"@anyssiakaissa depakote made me feel flat, but...",10016759.0,flat affect,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2240,347414282138431488,78.0,89.0,ADR,weight gain,zyprexa,"@anyssiakaissa depakote made me feel flat, but...",10047896.0,weight gain,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2242,348383854089871360,72.0,91.0,ADR,gain so much weight,zyprexa,that zyprexa really makes your vocal chords ma...,10047896.0,weight gain,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2244,349181683293102080,99.0,117.0,ADR,increase my weight,zyprexa,"i'm so fine today. increasing zyprexa,my condi...",10047898.0,weight increase,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def clean_tweet(tweet):
    tweet = re.sub(r"#", "", tweet, 0, re.MULTILINE)
    tweet = re.sub(r"rt", "", tweet, 0, re.MULTILINE)
    # tweet = tweet.lower()
    regex_mentions = r"@\w+"
    tweet = re.sub(regex_mentions, "@USER__", tweet, 0, re.MULTILINE)
#     regex_hash = r"(#[^\s]+)"
#     tweet = re.sub(regex_hash, "", tweet, 0, re.MULTILINE)
#     hash_matched = re.finditer(
#         regex_hash, tweet['full_text'], re.MULTILINE)
    url_regex = r"(http[^\s]+)"
    tweet = re.sub(url_regex, "", tweet, 0, re.MULTILINE)
    # special_char_re = r"[^(?u)\w]+"
    # tweet = re.sub(special_char_re, " ", tweet, 0, re.MULTILINE)
    return tweet

In [7]:
def emoji2Text(text):
    emojiTranslate = { # top 19 emojis used
    '😂': 'laughing',
    '😩': 'distraught',
    '😳': 'surprise',
    '💊': 'medication',
    '😭': 'laughing crying',
    '😒': 'disapproving',
    '😴': 'bored tired',
    '😷': 'sick',
    '😔': 'remorseful',
    '👌': 'correct',
    '👍': 'approve',
    '😫': 'weary',
    '🙌': 'celebration',
    '😏': 'smug',
    '😊': 'happy',
    '😖': 'confounded',
    '💤': 'tired',
    '😁': 'cheerful',
    '❤️': 'love'}

    for word, e in emojiTranslate.items():
        text = text.replace(word.lower(), e)
    
    text = text.lstrip().rstrip()
    
    return text

In [8]:
def remove_emojis(text):
    # emoticons_happy = list([
    #     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    #     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    #     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    #     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    #     '<3'
    # ])
    # emoticons_sad = list([
    #     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    #     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    #     ':c', ':{', '>:\\', ';('
    # ])
    all_emoticons = [':/',':l',':-)','-.-',':::---))))','&lt;',":'(",'-_-',':)','xx','xxx',';-)',
                     ':(',':-/',':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
                     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D','d:',':-l','&lt;3',
                     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',':|','o_o',';)',
                     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',':-0','-__-',':(((',
                     '<3'':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
                     ':-[', ':-<', '=/', '>:(', ':(', '>.<', ":'-(", ":'(",  ':-c',':c', ':{',  ';(', "=\\", ">:\\", ':\\']

    emojis = list(demoji.findall(text).keys())
    clean_text = demoji.replace(text, '')

    for emo in all_emoticons:
        if (emo in clean_text):
            clean_text = clean_text.replace(emo, '')
            emojis.append(emo)

    clean_text = clean_text.replace('\n', '', -1)
    
    # using regex to remove urls
    # regex_url = r"https:(.*)[A-Za-z0-9]"
    # clean_text = re.sub(regex_url, '', clean_text, 0, re.MULTILINE)

    return clean_text #, emojis

In [9]:
df['tweet'] = df['tweet'].map(lambda tweet: clean_tweet(remove_emojis(emoji2Text(tweet))))
df_eval['tweet'] = df_eval['tweet'].map(lambda tweet: clean_tweet(remove_emojis(emoji2Text(tweet))))

In [10]:
df

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,Unnamed: 9,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@USER__:oral drugs for pyelonephritisiprofloxa...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as a...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,@USER__ i'm ok ty have an official diagnosis o...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,time for my daily afternoon relaxation ritual ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,347921687729299456,NaN,NaN,NaN,NaN,NaN,i will admit that most people would agree zypr...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2242,348383854089871360,72.0,91.0,ADR,gain so much weight,zyprexa,that zyprexa really makes your vocal chords ma...,10047896.0,weight gain,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2243,348595096352075776,NaN,NaN,NaN,NaN,zyprexa,@USER__ i'm hoping that i get on the zyprexa. ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2244,349181683293102080,99.0,117.0,ADR,increase my weight,zyprexa,"i'm so fine today. increasing zyprexa,my condi...",10047898.0,weight increase,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_eval

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@USER__ if avelox has hu your liver, avoid tyl...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro @USER__: @USER__ i...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,@USER__ i was given like 5mg of abilify and i ...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN
